# Comparison of different methods to create dataset

Import dependencies

In [1]:
# Standard Library
import os, sys
import glob, shutil

# PyPI dependencies
import cv2
import chainercv
import imageio

import numpy as np
import ipywidgets as wgt
import matplotlib.pyplot as plt

from tqdm import tqdm
from time import time
from sklearn.manifold import TSNE
from sklearn import preprocessing
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from cvt.models import SubspaceMethod, KernelMSM
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Confirm we are working in the correct directory.
path = '/home/atom/emotion/' # change this to the correct path
if os.getcwd().split(os.sep)[-1] != 'emotion':
    os.chdir(path)
assert os.getcwd().split(os.sep)[-1] == 'emotion', 'please set the correct path. Current: {0}'.format(os.getcwd())

import atomity

# import JVAER tools
from JAVER import utils
from JAVER import scrape_tools
from JAVER import track_tools

In [2]:
import JAVER
import importlib

importlib.reload(JAVER.utils)
importlib.reload(JAVER.track_tools)
importlib.reload(JAVER.scrape_tools)
importlib.reload(JAVER.logger);

## Downloading images and videos

We download images using the google_images_download package and we download videos using the pytube package.
Pytube frequently breaks because of the frequent changes youtube makes and is not very stable. If it suddenly doesn't work I recommend checking their github repo for issues.

Estimate time for downloading,

- images <2 secs per image
- videos <1 min per video

I don't think this will become a bottle neck.

Also, a lot of this notebook uses ipython widgets quite a bit. Running this .ipynb in `jupyter notebook` (or `jupyter lab` if you have nbextensions set up) is recommended.

In [3]:
# Choose paths to save data
out_dir_path = './notebooks/single_person_results'

# Set arguments to run directory here
overwrite = True if input('overwrite existing files? y/n') == 'y' else False
query = input('Enter Query: ')

overwrite existing files? y/nn
Enter Query: Elon Musk


In [5]:
# Set up save directories
dirs1 = ['images', 'videos', 'embeddings', 'cropped_images', 'inliers', 'outliers', 'video_frames', 'new_videos']
JAVER.utils.make_save_dirs(out_dir_path, dirs1, [query], overwrite=overwrite)

n_images = int(input('Enter number of images to scrape: ')) # Number of images of each person to scrape
chromedriver_path = 'JAVER/chromedriver' # path to chromedriver

# get_face_images frequently fails to get n_images, it will usually get more than 80%
# if there are 0 images found it will recurse and try again.
with atomity.timing():
    JAVER.scrape_tools.get_face_images(query, n_images, os.path.join(out_dir_path, 'images'), chromedriver_path)

Enter number of images to scrape: 80


[2020-03-17 19:33:48,558] INFO @ line 66: Success: Loaded 80 images.


Time: 98.4616334438324


In [6]:
n_videos = 1 # Number of images of each person to scrape
with atomity.timing():
    video_paths = JAVER.scrape_tools.get_yt_videos(query, os.path.join(out_dir_path, 'videos'), n_videos)

[2020-03-17 19:33:55,160] INFO @ line 115: Successfully downloaded 1 videos.


Time: 6.588725805282593


In [7]:
paths = glob.glob('./notebooks/single_person_results/images/Elon Musk/*')
atomity.browse_images(paths);

interactive(children=(Dropdown(description='image_path', options=('./notebooks/single_person_results/images/El…

## Face detection

In [8]:
in_paths = glob.glob(r'./notebooks/single_person_results/images/*/*.jpg')
out_paths = [p.replace('{0}images{0}'.format(os.sep), '{0}cropped_images{0}'.format(os.sep)) for p in in_paths]

with atomity.timing():
    _ = utils.crop_faces(in_image_paths=in_paths, out_paths=out_paths)

[2020-03-17 19:33:55,220] INFO @ line 120: using: cuda:0


Time: 19.40379762649536


In [9]:
paths = glob.glob('./notebooks/single_person_results/cropped_images/Elon Musk/*')
atomity.browse_images(paths);

interactive(children=(Dropdown(description='image_path', options=('./notebooks/single_person_results/cropped_i…

## Face Embedding

In [10]:
in_paths = glob.glob(r'./notebooks/single_person_results/cropped_images/*/*.jpg')
out_paths = [p.replace('{0}cropped_images{0}'.format(os.sep), '{0}embeddings{0}'.format(os.sep)) for p in in_paths]

with atomity.timing():
    _ = utils.embed_faces(in_paths, out_paths, return_values=[])

[2020-03-17 19:34:14,677] INFO @ line 203: using: cuda:0


Time: 4.719070672988892


## Outlier Detection

Outlier detection does not seem to be effective enough...

In [11]:
embeddings = np.zeros((0,512))
embedding_paths = glob.glob(r'./notebooks/single_person_results/embeddings/{0}/*.npy'.format(query))
in_paths = glob.glob(r'./notebooks/single_person_results/cropped_images/{0}/*.jpg'.format(query))
assert len(embedding_paths) == len(in_paths)
for path in embedding_paths:
    embeddings = np.vstack((embeddings, np.load(path)))

inlier_idx, outlier_idx = JAVER.utils.detect_outliers(embeddings)

for idx in outlier_idx:
    shutil.copyfile(in_paths[idx], in_paths[idx].replace('cropped_images', 'outliers'))
    os.remove(embedding_paths[idx])# remove outlying embeddings

for idx in inlier_idx:
    shutil.copyfile(in_paths[idx], in_paths[idx].replace('cropped_images', 'inliers'))

[2020-03-17 19:34:19,508] INFO @ line 276: 10% are outliers


In [12]:
reference_image_paths = glob.glob('./notebooks/single_person_results/inliers/Elon Musk/*')
atomity.browse_images(reference_image_paths);

interactive(children=(Dropdown(description='image_path', options=('./notebooks/single_person_results/inliers/E…

In [13]:
paths = glob.glob('./notebooks/single_person_results/outliers/Elon Musk/*')
atomity.browse_images(paths);

interactive(children=(Dropdown(description='image_path', options=('./notebooks/single_person_results/outliers/…

## Facial Tracking

Facial tracking requires videos the videos that we have downloaded from youtube. For demonstration purposes we will use crop a video to about.. maybe 30 seconds?

In [155]:
# Standard Library
import os, sys
import glob, shutil

# PyPI dependencies
import cv2
import chainercv
import imageio

import numpy as np
import ipywidgets as wgt
import matplotlib.pyplot as plt

from tqdm import tqdm
from time import time
from sklearn.manifold import TSNE
from sklearn import preprocessing
from IPython.display import display
from PIL import Image, ImageDraw, ImageFont
from cvt.models import SubspaceMethod, KernelMSM
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

# Confirm we are working in the correct directory.
path = '/home/atom/emotion/' # change this to the correct path
if os.getcwd().split(os.sep)[-1] != 'emotion':
    os.chdir(path)
assert os.getcwd().split(os.sep)[-1] == 'emotion', 'please set the correct path. Current: {0}'.format(os.getcwd())

import atomity

# import JVAER tools
from JAVER import utils
from JAVER import scrape_tools
from JAVER import track_tools

import JAVER
import importlib

importlib.reload(JAVER.utils)
importlib.reload(JAVER.track_tools)
importlib.reload(JAVER.scrape_tools)
importlib.reload(JAVER.logger);

In [156]:
video_path = './notebooks/single_person_results/videos/Elon Musk/0.mp4'
cropped_video_path = os.path.join(os.path.dirname(video_path), 'cropped_video.mp4')

In [157]:
ffmpeg_extract_subclip(video_path, t1=150, t2=180, targetname=cropped_video_path)

video = imageio.get_reader(cropped_video_path,  'ffmpeg') # fails sometimes ? 
n_frames = video.count_frames()

# Make sure the subjects face is showing in consecutive frames
@wgt.interact(Frame=range(1, n_frames, 60))
def peek_video(Frame):
    display(Image.fromarray(video.get_data(Frame)))

Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful


interactive(children=(Dropdown(description='Frame', options=(1, 61, 121, 181, 241, 301, 361, 421, 481, 541, 60…

In [158]:
ref_paths = glob.glob('./notebooks/single_person_results/inliers/Elon Musk/*')
face_tracker = track_tools.FaceTracker(ref_paths = ref_paths, image_size=160, batch_size=100, step=100)

In [159]:
with atomity.timing():
    face_dict = face_tracker.detect(video_path=cropped_video_path)

[2020-03-21 18:39:17,561] DEBUG @ line 188: Frame: 0, n_faces: 2
[2020-03-21 18:39:17,692] DEBUG @ line 188: Frame: 100, n_faces: 2
[2020-03-21 18:39:17,812] DEBUG @ line 188: Frame: 200, n_faces: 2
[2020-03-21 18:39:17,875] DEBUG @ line 188: Frame: 300, n_faces: 1
[2020-03-21 18:39:17,940] DEBUG @ line 188: Frame: 400, n_faces: 1
[2020-03-21 18:39:17,941] DEBUG @ line 183: Frame: 500, n_faces: 0
[2020-03-21 18:39:17,944] DEBUG @ line 183: Frame: 600, n_faces: 0
[2020-03-21 18:39:17,945] DEBUG @ line 183: Frame: 700, n_faces: 0
[2020-03-21 18:39:17,947] DEBUG @ line 183: Frame: 800, n_faces: 0
[2020-03-21 18:39:17,947] INFO @ line 204: Rollback to 0
[2020-03-21 18:39:21,076] DEBUG @ line 188: Frame: 0, n_faces: 2
[2020-03-21 18:39:21,200] DEBUG @ line 188: Frame: 10, n_faces: 2
[2020-03-21 18:39:21,320] DEBUG @ line 188: Frame: 20, n_faces: 2
[2020-03-21 18:39:21,377] DEBUG @ line 188: Frame: 30, n_faces: 1
[2020-03-21 18:39:21,490] DEBUG @ line 188: Frame: 40, n_faces: 2
[2020-03-21 1

[2020-03-21 18:39:28,141] DEBUG @ line 188: Frame: 440, n_faces: 1
[2020-03-21 18:39:28,259] DEBUG @ line 188: Frame: 450, n_faces: 2
[2020-03-21 18:39:28,375] DEBUG @ line 188: Frame: 460, n_faces: 2
[2020-03-21 18:39:28,498] DEBUG @ line 188: Frame: 470, n_faces: 2
[2020-03-21 18:39:28,615] DEBUG @ line 188: Frame: 480, n_faces: 2
[2020-03-21 18:39:28,673] DEBUG @ line 188: Frame: 490, n_faces: 1
[2020-03-21 18:39:28,794] DEBUG @ line 188: Frame: 500, n_faces: 2
[2020-03-21 18:39:28,914] DEBUG @ line 188: Frame: 510, n_faces: 2
[2020-03-21 18:39:29,032] DEBUG @ line 188: Frame: 520, n_faces: 2
[2020-03-21 18:39:29,095] DEBUG @ line 188: Frame: 530, n_faces: 1
[2020-03-21 18:39:29,212] DEBUG @ line 188: Frame: 540, n_faces: 2
[2020-03-21 18:39:29,324] DEBUG @ line 188: Frame: 550, n_faces: 2
[2020-03-21 18:39:29,439] DEBUG @ line 188: Frame: 560, n_faces: 2
[2020-03-21 18:39:29,555] DEBUG @ line 188: Frame: 570, n_faces: 2
[2020-03-21 18:39:29,615] DEBUG @ line 188: Frame: 580, n_face

[2020-03-21 18:39:34,388] DEBUG @ line 183: Frame: 890, n_faces: 0
[2020-03-21 18:39:34,389] INFO @ line 214: Rollforward to 800
[2020-03-21 18:39:34,441] DEBUG @ line 183: Frame: 800, n_faces: 0


Time: 18.524126768112183


In [173]:
face_dict

{0: [<JAVER.track_tools.Face at 0x7f01fde72910>,
 10: [<JAVER.track_tools.Face at 0x7f01fd67b9a0>,
 20: [<JAVER.track_tools.Face at 0x7f0220177220>,
 30: [<JAVER.track_tools.Face at 0x7f01fde945e0>],
 40: [<JAVER.track_tools.Face at 0x7f01fde94d90>,
 50: [<JAVER.track_tools.Face at 0x7f01fde946d0>,
 60: [<JAVER.track_tools.Face at 0x7f01fde943d0>,
 70: [<JAVER.track_tools.Face at 0x7f01fde94f10>,
 80: [<JAVER.track_tools.Face at 0x7f01fde94dc0>],
 90: [<JAVER.track_tools.Face at 0x7f01fde94760>,
 100: [<JAVER.track_tools.Face at 0x7f01fde943a0>,
 110: [<JAVER.track_tools.Face at 0x7f024d27eeb0>,
 120: [<JAVER.track_tools.Face at 0x7f024d27e820>],
 130: [<JAVER.track_tools.Face at 0x7f024d27e3d0>,
 140: [<JAVER.track_tools.Face at 0x7f024d27ed00>,
 150: [<JAVER.track_tools.Face at 0x7f024d27e7c0>,
 160: [<JAVER.track_tools.Face at 0x7f024d27e220>,
 170: [<JAVER.track_tools.Face at 0x7f024d27e5b0>],
 180: [<JAVER.track_tools.Face at 0x7f024d27eee0>,
 190: [<JAVER.track_tools.Face at 0x7f

In [176]:
from time import sleep
for idx, faces in face_dict.items():
    for face in faces:
        I(face.face_img)
        sleep(3)


KeyboardInterrupt: 

In [7]:
def format_input(X, y):
    X = [X[np.where(y==t)] for t in np.unique(y)]
    return X, np.unique(y)
face
in_paths = glob.glob(r'./notebooks/single_person_results/embeddings/*/*.npy')

NameError: name 'MTCNN' is not defined

In [13]:
# load reference face
x, y = np.zeros((0,512)), []
for path in in_paths:
    label = 1 if os.path.basename(os.path.dirname(path)) == query else 0
    x = np.vstack((x, np.load(path)))
    y.append(label)

# use SM for face matching
SM = SubspaceMethod(n_subdims=5)
SM.fit(*format_input(x, y))

In [34]:
# Initialize face tracker
face_tracker = track_tools.FaceTracker(image_size=160, batch_size=100, step=100)

In [ ]:
# Get boundary boxes for each frame
bboxes = face_tracker.track(
    video_path=cropped_video_path, reference_images=reference_image_paths)

In [ ]:
tqdm._instances.clear()

# init loop
vid.set_image_index(0)

batch_size = 100
image_batch = []

# loop video framesa
for i in tqdm(range(0, n_frames, batch_size), total=n_frames):   
    
    image = vid.get_next_data()
    if i % batch_size == 0:
        image_batch.append(image)

    if len(image_batch) == batch_size:
        boxes, probs = track_tools.mtcnn(images=image_batch)
        
        if l := probs.co:
            print(probs)
            break
        image_batch = []
        
    for _ in range(batch_size):
        pass
    
    pass

#         image = Image.fromarray(im)
#         ret_dict = utils.crop_faces(images=[image], return_values=['bboxes', 'rois'])
        
#         bboxes = ret_dict['bboxes']
#         rois = ret_dict['rois']
#         embeddings = utils.embed_faces(images=rois, return_values='embeddings')['embeddings']
#         assert len(bboxes) == len(embeddings)

#         draw = ImageDraw.Draw(image)

#         for n in range(len(bboxes)):
            
#             bbox = bboxes[n]
#             x1,y1,x2,y2 =bboggggggggggggggggggggx
#             roi = rois[n]
#             embedding = embeddings[n]

#             proba = SM.predict_proba(embedding).squeeze()
#             text = 'proba: {0}\n {1}'.format(proba, 'Elon' if proba >= 0.7 else 'Not Elon')
#             font = ImageFont.truetype("/usr/share/fonts/truetype/freefont/FreeMono.ttf", 18, encoding="unic")

#             rect = (x1, y1, x1, y2, x2, y2, x2,y1, x1, y1)
#             draw.line(rect, width=5, fill='Red')
#             draw.text(xy=(x1, y1), text=text, fill='white', font=font)
            
#         image.save('./notebooks/single_person_results/video_frames/Elon Musk/' +str(i).zfill(6)+ ".png", "png")

In [ ]:
#         vid_writer.write(np.asarray(image))
#     if i!=0 and i % 1000 == 0:
#         break
# vid_writer.release()


#                 if prev_bbox is None:
#                     if len(embeddings) == 1:
                        
#                         if SM.predict(embeddings):
#                             text = 'proba: {0}, {1}'.format(proba, 'Elon' if proba < 0.8 else 'Not Elon')
#                     else:
#                         continue                
#                 else:                
#                     candidate_idx = np.where(SM.predict(embeddings)==1)
#                     candidate_rois = rois[candidate_idx]
#                     candidate_bboxes = bboxes[candidate_idx]
#                     print(candidate_bboxes)

            # TODO Add chunk to list
#             print('Mean of frame {0} is {1}, {2}, {3}'.format(i, im.mean(), type(im), im.shape))

            # TODO: face detection
            # Get list of all face bboxes in one frame
#             

            # TODO: Association with IoU and Embeddings


            # TODO Add chunk to list

        # TODO split vid to chunks


In [ ]:
image_folder = './notebooks/single_person_results/video_frames/Elon Musk'
video_name = './notebooks/single_person_results/movie.avi'

images = sorted([img for img in os.listdir(image_folder) if img.endswith(".png")])
frame = cv2.imread(os.path.join(image_folder, images[0]))
height, width, layers = frame.shape

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
video = cv2.VideoWriter(video_name, fourcc, 25, (width,height))

for i in tqdm(range(len(images))):
    image = images[i]
    video.write(cv2.imread(os.path.join(image_folder, image)))

cv2.destroyAllWindows()
video.release()

In [ ]:
paths = glob.glob('./notebooks/single_person_results/new_vid_frames/Elon Musk/*')
atomity.browse_images(paths);